In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import warnings
from sklearn.exceptions import DataConversionWarning

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

# Suppress DataConversionWarning messages
warnings.simplefilter(action='ignore', category=DataConversionWarning)


# EDA-- explanatory data analysis

In [3]:
data=pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [5]:
data["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

# lets handle the unbalanced dataset

In [6]:
data_maj=data[data["Class"]==0]
data_min=data[data["Class"]==1]

In [7]:
print(data_maj["Class"].value_counts())
print(data_min["Class"].value_counts())

Class
0    284315
Name: count, dtype: int64
Class
1    492
Name: count, dtype: int64


In [8]:
resample_size=len(data)-len(data_min)
resample_size

284315

In [9]:
data_min_upsampled=resample(data_min,replace=True,n_samples=resample_size,random_state=42)

In [10]:
data_balanced=pd.concat([data_maj,data_min_upsampled])
data_balanced.shape

(568630, 31)

In [11]:
data_balanced["Class"].value_counts()

Class
0    284315
1    284315
Name: count, dtype: int64

# lets train our Random Forest

In [12]:
x=data_balanced.drop(["Class"],axis=1)
y=data_balanced["Class"]
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)

In [13]:
# lets check the shape
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((426472, 30), (426472,), (142158, 30), (142158,))

In [14]:
model=RandomForestClassifier()
model.fit(x_train,y_train)

RandomForestClassifier()

In [15]:
from sklearn.metrics import classification_report,confusion_matrix
y_pred=model.predict(x_test)
print(confusion_matrix(y_pred,y_test))

[[71009     0]
 [    6 71143]]


In [16]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71009
           1       1.00      1.00      1.00     71149

    accuracy                           1.00    142158
   macro avg       1.00      1.00      1.00    142158
weighted avg       1.00      1.00      1.00    142158

